In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)

OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/output/"
OUT_DIR2 <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/output_summation/"

N_FOLDS <- 10

#DESeq2 PCA plot
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts.Rds", sep = ""))

vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
GEO_model_training_tissue_detail.vec <-
   readRDS(paste(OUT_DIR, "GEO_model_training_tissue_detail_vec.Rds", sep = ""))
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls.Rds", sep = ""))
rxns <- rxn2ensembls.nls %>% names()

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
print(head(vst.count.mtx, n = 5))

                          V1        V2        V3        V4        V5        V6
ENSMUSG00000000001 15.509603 12.614849 15.519749 14.637853 14.055204 13.449194
ENSMUSG00000000028  8.444428  7.614693 10.420513  9.853543  7.905628  8.286601
ENSMUSG00000000049 20.595033  6.507020  8.077450  7.916962 20.798987  6.163521
ENSMUSG00000000058 10.526181 11.266114  8.766421 11.650309  9.692308 15.196969
ENSMUSG00000000085  9.811443 13.441425 11.548222 12.198297 11.906675 12.810133
                          V7        V8       V9       V10       V11      V12
ENSMUSG00000000001 16.044583 15.004628 9.411033 13.508342 12.476597 14.78003
ENSMUSG00000000028 11.641435  8.189689 9.411033  9.384565  8.152655 10.04961
ENSMUSG00000000049  7.423413 18.564235 9.411033  7.550226 15.490085 19.81449
ENSMUSG00000000058  6.027590 10.630020 9.411033 14.140981  9.504138 10.47771
ENSMUSG00000000085 11.145885 11.276172 9.411033 11.846290 12.269073 11.78155
                         V13       V14       V15       V16      

ENSMUSG00000000028 12.309553 12.39087  9.530738 11.233997 10.638877  7.947121
ENSMUSG00000000049  6.013099 19.61428  6.845847  8.963214  4.376952  6.624554
ENSMUSG00000000058  8.883434 10.86095 17.662386 10.898600 13.571580 11.687129
ENSMUSG00000000085 14.293366 11.33846 11.605394 10.864526 10.472196 12.760652
                        V109      V110      V111      V112      V113      V114
ENSMUSG00000000001 15.665475 14.626659 12.219381  4.992806 14.342172 14.302321
ENSMUSG00000000028  7.543999  8.936425  8.709693  4.992806 11.351072  8.391464
ENSMUSG00000000049 19.944615  6.475052  7.451308  8.062140  8.604859 19.160273
ENSMUSG00000000058 10.579660 13.227303 12.195277  4.992806  4.197949  9.906386
ENSMUSG00000000085 10.015609 12.604300 13.672737 13.879190  9.806942 10.977229
                        V115      V116     V117      V118      V119      V120
ENSMUSG00000000001 14.890392 12.961149 15.63412 15.417743 13.222703  7.384598
ENSMUSG00000000028 12.827788  8.954466 10.72962  9.866863 

ENSMUSG00000000058 13.292905 12.651423 12.086299 17.918717 16.456457 10.899493
ENSMUSG00000000085 11.808747 13.460730 12.910845 11.336657 11.358636 11.961096
                       V211      V212      V213      V214      V215     V216
ENSMUSG00000000001 14.44785 15.270249 14.115075 12.532169 13.511649 14.95045
ENSMUSG00000000028  9.68242  8.780646  8.610881  7.605350 12.082206  8.46546
ENSMUSG00000000049  7.96584 19.729160 10.378436  7.706506  8.726329 20.56969
ENSMUSG00000000058 17.48494 10.412772 11.963935 12.548102 13.873666 10.19696
ENSMUSG00000000085 11.31401 10.990248 11.918301 12.661413 12.083736 10.71115
                        V217      V218      V219      V220      V221      V222
ENSMUSG00000000001 13.408858 12.040765 14.206700 14.764354 14.023730 15.083858
ENSMUSG00000000028  9.928899  9.824083  8.905803  8.558070  9.603014  8.564805
ENSMUSG00000000049  5.952458  7.851414 21.728610 20.817389  7.291994 14.852269
ENSMUSG00000000058 14.792544 12.166673  8.255470  9.377549  3.80

                        V313      V314      V315      V316      V317      V318
ENSMUSG00000000001 16.224108 18.383297 12.340661 14.484115 12.633390 14.838631
ENSMUSG00000000028 10.145784  9.284540  9.359052  8.228720 10.797646 12.407957
ENSMUSG00000000049  8.711811  8.421682  7.463809 21.216337  5.966518  7.484144
ENSMUSG00000000058 10.005762  3.915712 14.328300  9.053884 14.938358  9.568856
ENSMUSG00000000085 10.665400 11.836686 12.055454 11.489461 11.382169 13.357427
                        V319      V320      V321      V322      V323      V324
ENSMUSG00000000001  9.839379 13.219285 14.345012 15.298796 15.128658 15.158826
ENSMUSG00000000028  5.893797  9.528652 13.567012  6.399269  9.369363  9.023283
ENSMUSG00000000049  5.893797  5.914923  5.874627 20.120883  7.873365 12.003129
ENSMUSG00000000058  5.893797 11.541564 10.725998 10.017327 11.191646 13.684258
ENSMUSG00000000085 20.582405 11.219182 13.110358 11.131080 11.877728 12.032371
                        V325      V326      V327    

ENSMUSG00000000028 10.155970 15.550427  8.821842  8.34289 11.890498  7.883834
ENSMUSG00000000049  4.951455  7.453559 19.965126 20.60419 12.608715  6.991516
ENSMUSG00000000058 16.634591 14.397653  9.111862 10.18375

...

ENSMUSG00000000058 12.179406 13.312042  8.551790 9.582513 10.690190 11.190519
ENSMUSG00000000085 12.262027 14.420208  3.951793 9.582513 11.656680  9.670394
                      V5953     V5954     V5955     V5956     V5957     V5958
ENSMUSG00000000001 14.18865 15.200085 15.130857 16.270026 15.503290 15.724318
ENSMUSG00000000028  8.36426  8.764392  7.529137  6.581025  9.270309  8.054020
ENSMUSG00000000049 10.00188 20.480122  3.738327  6.581025 11.906453 20.718146
ENSMUSG00000000058 13.00781  9.711145 15.396746 21.895783 10.180389  9.989844
ENSMUSG00000000085 11.76981 11.168619  9.451783  6.581025 12.429493 10.153270
                       V5959     V5960     V5961     V5962     V5963    V5964
ENSMUSG00000000001 14.844731 15.256079 12.676177 13.994598 13.930629 14.97906
ENSMUSG00000000028  8.232627  8.793873 13.525463  8.190412  9.568615  8.16609
ENSMUSG00000000049 10.404539 19.712792 17.158481  6.117964  6.076168 20.31829
ENSMUSG00000000058 14.084527  9.810541  7.309178 12.098971 12.04

                       V6055    V6056     V6057     V6058     V6059    V6060
ENSMUSG00000000001 12.504843 10.03125 13.411581 12.715905 13.926740 12.95148
ENSMUSG00000000028  8.897309 10.03125  9.880511  9.345475  9.170565  9.08679
ENSMUSG00000000049  7.519783 10.03125  9.183135  5.652305  6.444929  8.53056
ENSMUSG00000000058 12.946036 10.03125  4.130948 13.807608 12.403620 14.76785
ENSMUSG00000000085 12.312656 22.12502 12.750399 13.455183 11.504571 12.94392
                       V6061     V6062    V6063     V6064     V6065     V6066
ENSMUSG00000000001 14.123153 12.101540 9.547426 15.704301 13.371120 13.812208
ENSMUSG00000000028  9.764853  7.665258 9.547426 10.432637 16.753137  8.538272
ENSMUSG00000000049  5.813293  6.220766 9.547426  8.350116  6.838082  6.143744
ENSMUSG00000000058 12.141471 11.938808 9.547426 10.023256 15.678063 15.722930
ENSMUSG00000000085 11.375387 12.344342 9.547426 10.662864 11.689295 12.395741
                       V6067     V6068     V6069     V6070     V6071  

ENSMUSG00000000028  9.286089  7.730258  8.036655  7.050809 10.412808 9.860032
ENSMUSG00000000049  6.261783 20.606659  7.791252  8.831036  8.194721 9.860032
ENSMUSG00000000058 16.285459 12.549190 11.837308 12.268817  3.897939 9.860032
ENSMUSG00000000085 11.995092 11.221255 12.420179 11.768600 12.508491 9.860032
                       V6163     V6164     V6165     V6166     V6167     V6168
ENSMUSG00000000001 14.909806 17.483408  3.886421 14.920421 15.042210 15.200306
ENSMUSG00000000028  9.953157  9.325051 10.250384  8.612666  8.565594  7.455079
ENSMUSG00000000049 20.474675  9.498269  7.629035 19.938677 20.282086 20.794095
ENSMUSG00000000058 10.235050 10.625149  3.886421  9.621876  9.974511  9.693198
ENSMUSG00000000085 10.138073  9.901479 12.449892 11.447161 10.577175 11.326121
                       V6169     V6170     V6171     V6172     V6173     V6174
ENSMUSG00000000001 14.363274 13.995470 14.353107 11.279746 15.333734 15.767460
ENSMUSG00000000028  8.483732  8.204509  8.998763 13.2879

ENSMUSG00000000058 16.559584 12.255174  7.831247 10.729830 16.953940 10.846900
ENSMUSG00000000085 11.155609 11.605950 11.034919 11.471040 11.293576  9.244326
                       V6265    V6266     V6267

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
%%R
print(head(GEO_model_training_tissue_detail.vec, n = 5))
print(head(rxn2ensembls.nls, n = 5))

[1] "Liver"     "Brain"     "Intestine" "Intestine" "Liver"    
$`R-MMU-450466`
 [1] "ENSMUSG00000000568" "ENSMUSG00000002102" "ENSMUSG00000018286"
 [4] "ENSMUSG00000021024" "ENSMUSG00000021178" "ENSMUSG00000021737"
 [7] "ENSMUSG00000021832" "ENSMUSG00000029440" "ENSMUSG00000030591"
[10] "ENSMUSG00000030603" "ENSMUSG00000030751" "ENSMUSG00000032869"
[13] "ENSMUSG00000040850" "ENSMUSG00000068749" "ENSMUSG00000069744"
[16] "ENSMUSG00000078652" "ENSMUSG00000079197"

$`R-MMU-450551`
[1] "ENSMUSG00000000568" "ENSMUSG00000022283" "ENSMUSG00000090877"
[4] "ENSMUSG00000091971"

$`R-MMU-450580`
[1] "ENSMUSG00000000568" "ENSMUSG00000022283" "ENSMUSG00000090877"
[4] "ENSMUSG00000091971"

$`R-MMU-72103`
 [1] "ENSMUSG00000000568" "ENSMUSG00000002658" "ENSMUSG00000006498"
 [4] "ENSMUSG00000007850" "ENSMUSG00000018379" "ENSMUSG00000019738"
 [7] "ENSMUSG00000021018" "ENSMUSG00000021546" "ENSMUSG00000024097"
[10] "ENSMUSG00000028330" "ENSMUSG00000029250" "ENSMUSG00000029538"
[13] "ENSMUSG00000030795" "

In [5]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [6]:
%%R
toi_indices <- seq(1,length(GEO_model_training_tissue_detail.vec))

# filter annotations
GEO_model_training_tissue_detail_vec_tis_of_interest <-
   GEO_model_training_tissue_detail.vec[toi_indices]

# filter expression data
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]

training_indices <-
   caret::createDataPartition(
      GEO_model_training_tissue_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )

vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

GEO_model_training_tissue_detail.vec.train <-
   GEO_model_training_tissue_detail_vec_tis_of_interest[training_indices]
GEO_model_training_tissue_detail.vec.test <-
   GEO_model_training_tissue_detail_vec_tis_of_interest[-training_indices]

saveRDS(GEO_model_training_tissue_detail.vec.train,file=paste(OUT_DIR2,"GEO_model_training_tissue_detail_vec_train.Rds",sep=""))

cv_fold_indices <- caret::createFolds(GEO_model_training_tissue_detail.vec.train,
                                      k = N_FOLDS)
binary_GEO_model_training_tissue_annotations <- unique(GEO_model_training_tissue_detail.vec)

full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [7]:
%%R
#perform this loop first
n_rxns <- length(rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   slice <- vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]
   rxn_pca <- if(class(slice) != "numeric"){colSums(slice)}else{slice}
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,".Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] "Processed 100 of 7856 reactions (1.3%)..."
[1] "Processed 200 of 7856 reactions (2.6%)..."
[1] "Processed 300 of 7856 reactions (3.8%)..."
[1] "Processed 400 of 7856 reactions (5.1%)..."
[1] "Processed 500 of 7856 reactions (6.4%)..."
[1] "Processed 600 of 7856 reactions (7.7%)..."
[1] "Processed 700 of 7856 reactions (8.9%)..."
[1] "Processed 800 of 7856 reactions (10.2%)..."
[1] "Processed 900 of 7856 reactions (11.5%)..."
[1] "Processed 1000 of 7856 reactions (12.7%)..."
[1] "Storing PCA objects containing reactions 0-1000 of 7856 reactions (12.7%)..."
[1] "Processed 1100 of 7856 reactions (14%)..."
[1] "Processed 1200 of 7856 reactions (15.3%)..."
[1] "Processed 1300 of 7856 reactions (16.6%)..."
[1] "Processed 1400 of 7856 reactions (17.8%)..."
[1] "Processed 1500 of 7856 reactions (19.1%)..."
[1] "Processed 1600 of 7856 reactions (20.4%)..."
[1] "Processed 1700 of 7856 reactions (21.7%)..."
[1] "Processed 1800 of 7856 reactions (22.9%)..."
[1] "Processed 1900 of 7856 reactio

In [8]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR2,"rxn_id_2_result_file_idx_nls.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR2, "full_rxn_pca_results_nls.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR2, "rxn_pca_nls.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR2, "vst_count_mtx_train.Rds", sep = ""))

In [9]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0
   
   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      GEO_model_training_tissue_detail.vec.train.cv_train <-
         GEO_model_training_tissue_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      GEO_model_training_tissue_detail.vec.train.cv_test <-
         GEO_model_training_tissue_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
      binary_GEO_model_training_tissue_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_GEO_model_training_tissue_annotations) {
         binary_GEO_model_training_tissue_detail_vec.test.cv_test_list[[tissue_annotation]] <-
            (GEO_model_training_tissue_detail.vec.train.cv_test == tissue_annotation)
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = GEO_model_training_tissue_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            GEO_model_training_tissue_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_GEO_model_training_tissue_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         
         tab <- table(cur_rxn_knn_calls,
                      binary_GEO_model_training_tissue_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_GEO_model_training_tissue_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-6798748 : Last ARI =  0.932812834238019 : Last ECOUNT =  55 : Last Lung MISCLASS =  0.00333864161004714 : Last Uterus MISCLASS =  0.00111465452666072 . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9840884 : Last ARI =  0.563438784977034 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.032122719008372 : Last Uterus MISCLASS =  0.00604236659974005 . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-2002466 : Last ARI =  0.568827950627364 : Last ECOUNT =  5 : Last Lung MISCLASS =  0.0254400985329623 : Last Uterus MISCLASS =  0.00556441448870959 . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-2247925 : Last ARI =  0.195308131180437 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0722016850943843 : Last Uterus MISCLASS =  0.0144745662935967 . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9016782 : Last ARI =  0.195257413093143 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0722016850943843 : Last Uterus MISCLASS =  0.0144745662935967 . Now  99.4 % remai

[1] "Last RXN_ID =  R-MMU-8850527 : Last ARI =  0.748277783843302 : Last ECOUNT =  7 : Last Lung MISCLASS =  0.0108176952281766 : Last Uterus MISCLASS =  0.00413558425819044 . Now  94.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-1604732 : Last ARI =  0.530174308619139 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0602738841455647 : Last Uterus MISCLASS =  0.0122447444624807 . Now  94.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-2213200 : Last ARI =  0.801515150356627 : Last ECOUNT =  4 : Last Lung MISCLASS =  0.0264053970727269 : Last Uterus MISCLASS =  0.00540339863506811 . Now  94.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-8948234 : Last ARI =  0.939679872851746 : Last ECOUNT =  17 : Last Lung MISCLASS =  0.00286422566218848 : Last Uterus MISCLASS =  0.000953392772351891 . Now  94.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-913529 : Last ARI =  0.0378828987377555 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0628249653275887 : Last Uterus MISCLASS =  0.0146355716340012 . Now  94.3 % rem

[1] "Last RXN_ID =  R-MMU-158484 : Last ARI =  0.860403943350665 : Last ECOUNT =  21 : Last Lung MISCLASS =  0.00540896735164219 : Last Uterus MISCLASS =  0.00127262761432307 . Now  89.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-450595 : Last ARI =  0.251585179678798 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.0537641707902743 : Last Uterus MISCLASS =  0.00985949321369433 . Now  89.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-5578744 : Last ARI =  0.85440872596897 : Last ECOUNT =  21 : Last Lung MISCLASS =  0.00572516671454706 : Last Uterus MISCLASS =  0.00206931390242168 . Now  89.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-70286 : Last ARI =  0.260058359988702 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0672752171101014 : Last Uterus MISCLASS =  0.0130487796058606 . Now  89.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-8852136 : Last ARI =  0.706997051649048 : Last ECOUNT =  6 : Last Lung MISCLASS =  0.013203454409552 : Last Uterus MISCLASS =  0.00190754423085637 . Now  89.2 % remaini

[1] "Last RXN_ID =  R-MMU-8982020 : Last ARI =  0.882724284664775 : Last ECOUNT =  9 : Last Lung MISCLASS =  0.00524542312482744 : Last Uterus MISCLASS =  0.00191083772345704 . Now  84.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-165162 : Last ARI =  0.241958734491862 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0580443469146471 : Last Uterus MISCLASS =  0.0112910985486198 . Now  84.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9693250 : Last ARI =  0.28644151451443 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.054710991912673 : Last Uterus MISCLASS =  0.00954303989791705 . Now  84.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-170844 : Last ARI =  0.363125929273553 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0516842547240394 : Last Uterus MISCLASS =  0.0136761070537485 . Now  84.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-2160932 : Last ARI =  0.819321336577609 : Last ECOUNT =  7 : Last Lung MISCLASS =  0.00429557861700273 : Last Uterus MISCLASS =  0.000635930047694755 . Now  84.1 % remaini

[1] "Last RXN_ID =  R-MMU-382613 : Last ARI =  0.596848378980403 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.035939572334972 : Last Uterus MISCLASS =  0.00492923989322586 . Now  79.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-2161792 : Last ARI =  0.708217627987961 : Last ECOUNT =  5 : Last Lung MISCLASS =  0.0170137408853821 : Last Uterus MISCLASS =  0.00906305284169352 . Now  79.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-983194 : Last ARI =  0.792423578560121 : Last ECOUNT =  4 : Last Lung MISCLASS =  0.0165443978074578 : Last Uterus MISCLASS =  0.00493202383810377 . Now  79.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-5696220 : Last ARI =  0.135521717998938 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0655117245618657 : Last Uterus MISCLASS =  0.0141545695276859 . Now  79.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-198746 : Last ARI =  0.69802095644529 : Last ECOUNT =  4 : Last Lung MISCLASS =  0.0125652362521072 : Last Uterus MISCLASS =  0.00636059363630666 . Now  79 % remaining...

...

[1] "Last RXN_ID =  R-MMU-443777 : Last ARI =  0.356260431526036 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0601118636859253 : Last Uterus MISCLASS =  0.013203201218102 . Now  23.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-139941 : Last ARI =  0.290482500783985 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0644127667759985 : Last Uterus MISCLASS =  0.0114521127795102 . Now  23.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-416683 : Last ARI =  0.372599790334903 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0217829759181925 : Last Uterus MISCLASS =  0.0133548380779229 . Now  23.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-5159250 : Last ARI =  0.0451140718866535 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0704477989368182 : Last Uterus MISCLASS =  0.0151071980073062 . Now  23.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-879575 : Last ARI =  0.631202434671597 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.0491379868524495 : Last Uterus MISCLASS =  0.011769330376434 . Now  23 % remaining..."
[1

[1] "Last RXN_ID =  R-MMU-110360 : Last ARI =  0.0394162081982726 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0686989953509381 : Last Uterus MISCLASS =  0.0163778019663792 . Now  18.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-73931 : Last ARI =  0.0900179327224663 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0666337094668582 : Last Uterus MISCLASS =  0.012412342440761 . Now  18.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-443774 : Last ARI =  0.159983528064741 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.066952186422504 : Last Uterus MISCLASS =  0.0144727836861422 . Now  18.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-8985591 : Last ARI =  0.199018366999372 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0632781021514339 : Last Uterus MISCLASS =  0.0139983911103797 . Now  18 % remaining..."
[1] "Last RXN_ID =  R-MMU-2029468 : Last ARI =  0.0543068303431334 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0698141280086869 : Last Uterus MISCLASS =  0.0130361208280911 . Now  17.9 % remaining..."


[1] "Last RXN_ID =  R-MMU-74733 : Last ARI =  0.112213974615456 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0690268175266849 : Last Uterus MISCLASS =  0.0136763554000663 . Now  13.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-2730896 : Last ARI =  0.130282133180313 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0656894602547522 : Last Uterus MISCLASS =  0.0147892329642807 . Now  13.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-73620 : Last ARI =  0.501442494362596 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0653577979810553 : Last Uterus MISCLASS =  0.0171772745989352 . Now  13.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-5358484 : Last ARI =  0.211466730940487 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0532728157275724 : Last Uterus MISCLASS =  0.0128784073267976 . Now  12.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-140842 : Last ARI =  0.499599479804111 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0534274831945566 : Last Uterus MISCLASS =  0.0133591506620015 . Now  12.8 % remaining..."


[1] "Last RXN_ID =  R-MMU-1655453 : Last ARI =  0.244853729836229 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0683762098334221 : Last Uterus MISCLASS =  0.0138330618804986 . Now  8.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-8870465 : Last ARI =  0.154189073471239 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0702938490454132 : Last Uterus MISCLASS =  0.0132009155347022 . Now  8.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-1678944 : Last ARI =  0.0360416037479422 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0629859489493866 : Last Uterus MISCLASS =  0.0138396319614873 . Now  8 % remaining..."
[1] "Last RXN_ID =  R-MMU-983702 : Last ARI =  0.0500686457690428 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0494620196504288 : Last Uterus MISCLASS =  0.0147907551509552 . Now  7.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-212005 : Last ARI =  0.646035920806795 : Last ECOUNT =  5 : Last Lung MISCLASS =  0.00747727585970146 : Last Uterus MISCLASS =  0.0090650821611297 . Now  7.7 % remaining..."
[

[1] "Last RXN_ID =  R-MMU-6811508 : Last ARI =  0.0409642138718676 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0739472080810088 : Last Uterus MISCLASS =  0.0125662537170864 . Now  3 % remaining..."
[1] "Last RXN_ID =  R-MMU-450325 : Last ARI =  0.065320043254435 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0644061934842383 : Last Uterus MISCLASS =  0.0130422039068059 . Now  2.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-3446785 : Last ARI =  0.169652680647349 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0602660498913872 : Last Uterus MISCLASS =  0.014152043565506 . Now  2.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5688294 : Last ARI =  0.0452873369986567 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0640824105476331 : Last Uterus MISCLASS =  0.015105933803501 . Now  2.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-112033 : Last ARI =  0.290317001780479 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0650527561817868 : Last Uterus MISCLASS =  0.0152641608631893 . Now  2.5 % remaining..."
[1] 

In [10]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR2, "toi_rxn_knn_misclass_rate_nls.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR2, "toi_rxn_knn_ari_nls.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR2, "toi_rxn_knn_ecount_nls.Rds", sep = ""))

d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)

saveRDS(d, paste(OUT_DIR2, "toi_summary_df.Rds", sep = ""))

In [11]:
%%R
library(htmlwidgets)
# generate sample figures
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[["R-MMU-450466"]], ]), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = GEO_model_training_tissue_detail.vec.train
)
ggplot(pca.d) +
   geom_point(aes(x = PC1, y = PC2, colour = Section)) +
   theme_bw()
p <- plot_ly(
   x = pca.d$PC1,
   y = pca.d$PC2,
   z = pca.d$PC3,
   type = "scatter3d",
   mode = "markers",
   color = pca.d$Section,
   size = 1
)
saveWidget(p, "summation_plotly_chart.html", selfcontained = TRUE)

In addition: Warning messages:
1: In RColorBrewer::brewer.pal(N, "Set2") :
  n too large, allowed maximum for palette Set2 is 8
Returning the palette you asked for with that many colors

2: In RColorBrewer::brewer.pal(N, "Set2") :
  n too large, allowed maximum for palette Set2 is 8
Returning the palette you asked for with that many colors

